Imports

In [1]:
import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice and easy way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

More imports to load mnist

In [2]:
import torchvision
from torch.utils.data import DataLoader # data management 
import torchvision.datasets as datasets # standard datasets
import torchvision.transforms as transforms # data processing

Set up MNIST and batch size

In [3]:
batch_size = 64
train_dataset = datasets.MNIST( 
    root="dataset/",
    train=True,
    transform=transforms.ToTensor(),
    download=True,
)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_dataset = datasets.MNIST(
    root="dataset/",
    train=False,
    transform=transforms.ToTensor(),
    download=True,
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

Set up base traning class


In [4]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x 

Make it a CNN

In [6]:
class CNN(nn.Module):
    def __init__(self, input_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.Conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride=(1,1), padding = (1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))
        self.Conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride=(1,1), padding = (1,1))
        self.fc = nn.Linear(16*7*7, num_classes)
        
    def forward(self,x):
        x = F.relu(self.Conv1(x))
        x = self.pool(x)
        x = F.relu(self.Conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x

# Setting up the training loop

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_channels = 1
num_classes = 10
learning_rate = 0.001
num_epochs = 3

In [8]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # No need toget to correct shape, 28x28->784
        # -1 will flatten all outer dimensions into one
        # data = data.reshape(data.shape[0], -1) 

        # forward propagation
        scores = model(data)
        loss = criterion(scores, targets)

        # zero previous gradients
        optimizer.zero_grad()
        
        # back-propagation
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

Epoch: 0
Epoch: 1
Epoch: 2


In [13]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct) / float(num_samples) * 100:.2f}"
        )

    model.train()

In [14]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 58934 / 60000 with accuracy 98.22
Got 9823 / 10000 with accuracy 98.23
